In [45]:
import pickle
from datetime import datetime
from os import mkdir
from os.path import join, exists

import numpy as np
import pandas as pd
import scipy as sp

# import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from lightgbm import LGBMClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from mlxtend.classifier import StackingCVClassifier

In [46]:
train_features = pd.read_pickle('../dataset/all_feature.pkl').reset_index(drop=True)
train_features.shape

(1080976, 282)

In [47]:
model_path = '../dataset/lightgbmmodel.pkl'

In [48]:
luck = 123456

In [49]:
model = LGBMClassifier(nthread=4, silent=False, objective= 'binary', seed=luck)

In [50]:
parameters = {
#     'gamma': [0.05],
    #     'n_estimators': [800],
#     'max_depth': [2, 6, 12],
    'max_depth': [12],
    'num_leaves': range(30, 50, 4), 

#     'learning_rate': [0.15, 0.3],
#         'subsample': [0.9],
#         'colsample_bytree': [0.9],
    #     'reg_alpha': [0, 1, 5],
    #     'reg_lambda': [0, 1, 4],
}

In [51]:
use_columns = [
    x for x in train_features.columns
    if x not in [
        'orderid', 'uid', 'hotelid', 'basicroomid', 'hotel_roomid', 'roomid',
        'orderlabel'
    ]
]

In [52]:
cv = model_selection.ShuffleSplit(n_splits=1, test_size=0.2, random_state=luck)
clf = model_selection.GridSearchCV(model, parameters, cv=cv, n_jobs=-1, verbose=True)

In [53]:
clf_feture = train_features.sample(100000, random_state=luck)

In [54]:
%time clf.fit(clf_feture[use_columns], clf_feture['orderlabel'])

Fitting 1 folds for each of 5 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   16.6s finished


CPU times: user 11.7 s, sys: 365 ms, total: 12.1 s
Wall time: 20.3 s


GridSearchCV(cv=ShuffleSplit(n_splits=1, random_state=123456, test_size=0.2, train_size=None),
       error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=5, min_split_gain=0, n_estimators=10, nthread=4,
        num_leaves=31, objective='binary', reg_alpha=0, reg_lambda=0,
        seed=123456, silent=False, subsample=1, subsample_for_bin=50000,
        subsample_freq=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [12], 'num_leaves': range(30, 50, 4)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [55]:
clf.best_score_

0.97294999999999998

In [56]:
clf.best_params_

{'max_depth': 12, 'num_leaves': 46}

In [75]:
2**6

64

In [76]:
# # clf.best_params_['n_estimators'] = 
clf.best_params_['max_depth'] = 6
# clf.best_params_['reg_lambda'] = 0.5
clf.best_params_['subsample'] = 0.8
clf.best_params_['colsample_bytree'] = 0.8
# clf.best_params_['n_estimators'] = 1000
clf.best_params_['learning_rate'] = 0.1
clf.best_params_['num_leaves'] = 60
# clf.best_params_['reg_alpha'] = 1
# clf.best_params_['min_data_in_leaf'] = 3

# clf.best_params_['min_child_weight'] = 1

In [77]:
model = LGBMClassifier(
    nthread=4,
    silent=False,
    objective='binary',
    seed=luck,
#     learning_rate=0.3,
    n_estimators=4000,
    **clf.best_params_)
model

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.8, learning_rate=0.1,
        max_bin=255, max_depth=6, min_child_samples=10, min_child_weight=5,
        min_split_gain=0, n_estimators=4000, nthread=4, num_leaves=60,
        objective='binary', reg_alpha=0, reg_lambda=0, seed=123456,
        silent=False, subsample=0.8, subsample_for_bin=50000,
        subsample_freq=1)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(
    train_features[use_columns],
    train_features['orderlabel'],
    test_size=0.33,
    random_state=42)

In [79]:
model.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    early_stopping_rounds=100,
    eval_metric='auc',
    verbose=True
    )

[1]	training's auc: 0.897172	valid_1's auc: 0.890173
Train until valid scores didn't improve in 100 rounds.
[2]	training's auc: 0.911038	valid_1's auc: 0.90594
[3]	training's auc: 0.912208	valid_1's auc: 0.907868
[4]	training's auc: 0.912216	valid_1's auc: 0.908057
[5]	training's auc: 0.912284	valid_1's auc: 0.908194
[6]	training's auc: 0.913538	valid_1's auc: 0.909078
[7]	training's auc: 0.914969	valid_1's auc: 0.910817
[8]	training's auc: 0.91608	valid_1's auc: 0.911812
[9]	training's auc: 0.916449	valid_1's auc: 0.912689
[10]	training's auc: 0.916536	valid_1's auc: 0.912676
[11]	training's auc: 0.916747	valid_1's auc: 0.912782
[12]	training's auc: 0.916944	valid_1's auc: 0.913209
[13]	training's auc: 0.917514	valid_1's auc: 0.913596
[14]	training's auc: 0.917388	valid_1's auc: 0.913394
[15]	training's auc: 0.917807	valid_1's auc: 0.913882
[16]	training's auc: 0.918518	valid_1's auc: 0.914492
[17]	training's auc: 0.918624	valid_1's auc: 0.914394
[18]	training's auc: 0.919068	valid_1'

LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.8, learning_rate=0.1,
        max_bin=255, max_depth=6, min_child_samples=10, min_child_weight=5,
        min_split_gain=0, n_estimators=4000, nthread=4, num_leaves=60,
        objective='binary', reg_alpha=0, reg_lambda=0, seed=123456,
        silent=False, subsample=0.8, subsample_for_bin=50000,
        subsample_freq=1)

In [80]:
def get_score(model, X_train, train_features):
    result = model.predict_proba(X_train)[:, 1]
    train_features['prob'] = np.nan
    train_features.loc[X_train.index, 'prob'] = result
    train_x = train_features.loc[X_train.index, ['orderid', 'prob', 'orderlabel']]
    train_x.sort_values('prob', ascending=False, inplace=True)
    finall_result_x = train_x.drop_duplicates(['orderid'])
    train_score = finall_result_x.orderlabel.mean()
    print(train_score)
    return train_score

In [81]:
train_score = get_score(model, X_train, train_features)

0.358606489323


In [82]:
test_score = get_score(model, X_test, train_features)

0.209509057849


In [83]:
if not exists('models'):
    mkdir('models')
model_importance_path = join('models', datetime.now().strftime('%d-%H%M-importance.txt'))
model_importance_path

'models/04-0731-importance.txt'

In [84]:
importance_df = pd.DataFrame(model.feature_importances_, index=use_columns)

importance_df.sort_values(0, ascending=False, inplace=True)
importance_df.to_csv(model_importance_path)

In [85]:
parms = ','.join(['{}: {}'.format(k, v) for k, v in clf.best_params_.items()])
parms

'subsample: 0.8,max_depth: 6,colsample_bytree: 0.8,learning_rate: 0.1,num_leaves: 60'

In [86]:
model.best_score

defaultdict(dict,
            {'training': {'auc': 0.94803508075567711},
             'valid_1': {'auc': 0.92719231720801976}})

In [87]:
print(datetime.now().strftime('%d-%H-%M:'),
      'online', '??',
     'test_score', test_score,
     'train_score', train_score,
     'model_train', model.best_score['training']['auc'],
     'model_test', model.best_score['valid_1']['auc'],
     'n', model.best_iteration,
     'params', parms, end='\n', sep='  ')

04-07-31:  online  ??  test_score  0.209509057849  train_score  0.358606489323  model_train  0.948035080756  model_test  0.927192317208  n  138  params  subsample: 0.8,max_depth: 6,colsample_bytree: 0.8,learning_rate: 0.1,num_leaves: 60


In [88]:
print(datetime.now().strftime('%d-%H-%M:'),
      'online', '??',
     'test_score', test_score,
     'train_score', train_score,
     'model_train', model.best_score['training']['auc'],
     'model_test', model.best_score['valid_1']['auc'],
     'n', model.best_iteration,
      'shape', train_features.shape,
     'params', parms, end='\n', sep=',', file=open('lgb_result.txt', 'a+'))

In [89]:
pickle.dump(model, open(model_path, 'wb'))

In [90]:
# clf1 =  LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=42, max_depth=6)
# clf2 = LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=32, max_depth=5)
# clf3 = LGBMClassifier(
#     nthread=4,
#     silent=False,
#     objective='binary',
#     seed=luck,
#     n_estimators=400,
#     num_leaves=27, max_depth=3)
# lr = LogisticRegression()
# sclf = StackingCVClassifier(classifiers=[clf1, clf2, clf3], 
#                           meta_classifier=lr)

# model = sclf